## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

In [0]:
from zipfile import ZipFile
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [0]:
import pandas as pd

In [3]:
import random

from tqdm import tqdm_notebook, tnrange
#from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
#from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Flatten
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load Dataset Files

In [0]:
labels = pd.read_csv('/content/drive/My Drive/R8_CV_Project2/labels.csv')

In [6]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [7]:
pd.unique(labels.breed).size

120

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/R8_CV_Project2/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
import os
#os.chdir('/content/drive/My Drive/R8_CV_Project2/train')

from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
#os.chdir('/content/drive/My Drive/R8_CV_Project2/test')

from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
#not required

Repeat the same step for labels.csv.zip

In [0]:
#not required

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [13]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [14]:
label_count = labels.breed.value_counts()
print(label_count)

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
komondor                 67
brabancon_griffon        67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64


### Get one-hot encodings of labels

In [0]:
from sklearn.preprocessing import OneHotEncoder 
  
onehotencoder = OneHotEncoder() 
data = onehotencoder.fit_transform(np.array(labels['breed']).reshape(-1, 1)).toarray()

In [0]:
data_ohe = pd.DataFrame(data)

In [0]:
data_ohe.columns = onehotencoder.categories_

In [18]:
data_ohe.head()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,english_foxhound,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
import os
basepath2 = '/content/train/'
files2 = []
for entry in os.listdir(basepath2):
    if os.path.isfile(os.path.join(basepath2, entry)):
        files2.append(entry)

#if fle.is_dir() is False:
#    with train_imgArchive.open(fle) as file:
#            img = Image.open(file)
#            #PIL image is saved as an png image, to convert it in to an array, we use np.array(image)
#            x_train.append(np.array(img))

In [20]:
len(files2)

10222

In [0]:
im_width = 128
im_height = 128

In [0]:
# Initializing an empty array for storing the image arrays
X = np.zeros((len(files2), im_height, im_width, 3), dtype=np.float32)
#y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)

In [0]:
label_count=0
for fle_name in labels.id:
  img = load_img(basepath2 + fle_name+'.jpg')
  x_img = img_to_array(img)
  x_img = resize(x_img, (128, 128, 3))
  X[label_count] = x_img/255
  label_count = label_count+1

In [24]:
print('Number of images read: ', label_count)

Number of images read:  10222


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [25]:
# data is already normalized in the above step.
print('Normalized Data: \n',X[0])

Normalized Data: 
 [[[0.6329613  0.416235   0.23339635]
  [0.6546272  0.48126888 0.27316448]
  [0.72479945 0.5730192  0.33029   ]
  ...
  [0.8743123  0.75024456 0.4383268 ]
  [0.84531367 0.73586744 0.34079024]
  [0.74472696 0.6219113  0.29560798]]

 [[0.49541402 0.3334337  0.19406752]
  [0.73178804 0.5922641  0.43648228]
  [0.7094787  0.569876   0.4016111 ]
  ...
  [0.79920834 0.63772583 0.3418426 ]
  [0.8415002  0.70436615 0.3785822 ]
  [0.8570511  0.7222736  0.32488644]]

 [[0.5723346  0.40296352 0.2310809 ]
  [0.60172164 0.43072847 0.25283983]
  [0.57591003 0.3905361  0.22937353]
  ...
  [0.74658734 0.5496322  0.21058485]
  [0.74975026 0.59366775 0.29637325]
  [0.8091298  0.6526616  0.25705042]]

 ...

 [[0.7382106  0.46416953 0.29341644]
  [0.73367476 0.47818902 0.2662934 ]
  [0.7205014  0.48173937 0.26516196]
  ...
  [0.62749165 0.35807723 0.27470735]
  [0.60369873 0.35637015 0.23437205]
  [0.7071898  0.5005505  0.33869168]]

 [[0.72698545 0.4645318  0.25721878]
  [0.7377348  0.48

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
X_train,X_val,y_train,y_val = train_test_split(X, data_ohe, train_size = 0.8, random_state = 123)

In [27]:
print(X_train.shape)
print(y_train.shape)

(8177, 128, 128, 3)
(8177, 120)


In [28]:
print(X_val.shape)
print(y_val.shape)

(2045, 128, 128, 3)
(2045, 120)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
sample_sub = pd.read_csv('/content/drive/My Drive/R8_CV_Project2/sample_submission.csv')

In [0]:
test_img = sample_sub.id

Run the below code to load the test image files in x_test_feature

In [0]:
#x_test_feature = []
#i = 0 # initialisation
#for f in tqdm(test_img.values): # f for format ,jpg
#    img = cv2.imread('./test/{}.jpg'.format(f), 0)
#    img_resize = cv2.resize(img, (img_rows, img_cols)) 
#    x_test_feature.append(img_resize)

#mport os
basepath = '/content/test/'
files = []
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)):
        files.append(entry)

In [0]:
x_test_feature = np.zeros((len(files), im_height, im_width, 3), dtype=np.float32)

In [0]:
tst_label_count=0
for tst_fle_name in test_img:
  img_tst = load_img(basepath + tst_fle_name+'.jpg')
  x_tst_img = img_to_array(img_tst)
  x_tst_img = resize(x_tst_img, (128, 128, 3))
  x_test_feature[label_count] = x_tst_img/255
  tst_label_count = tst_label_count+1

Normalize the test data and convert it into 4 dimensions

In [34]:
len(test_img)

10357

In [0]:
#Data is already normalized while reading the data in the above step.

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
import tensorflow as tf
from keras.models import Sequential
import keras

In [0]:
tf.keras.backend.clear_session()
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Conv2D(input_shape=(128,128,3),filters=64,kernel_size=(5,5),activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(units=256,activation="relu"))
model.add(Dense(units=120, activation="softmax"))

In [0]:
# Loss and Optimizer
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
callback_list = [early_stopping]# [stats, early_stopping]

### Use batch_size = 128 and epochs = 10 and execute the model

In [45]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=128, validation_data=(X_val, y_val), callbacks=callback_list, verbose=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 5s 553us/step - loss: 0.0166 - acc: 0.9994 - val_loss: 9.3290 - val_acc: 0.0631
Epoch 2/10
8177/8177 [==============================] - 4s 532us/step - loss: 0.0164 - acc: 0.9993 - val_loss: 9.3711 - val_acc: 0.0631
Epoch 3/10
8177/8177 [==============================] - 4s 532us/step - loss: 0.0120 - acc: 0.9996 - val_loss: 9.4723 - val_acc: 0.0636
Epoch 4/10
8177/8177 [==============================] - 4s 533us/step - loss: 0.0146 - acc: 0.9993 - val_loss: 9.4986 - val_acc: 0.0631
Epoch 5/10
8177/8177 [==============================] - 4s 533us/step - loss: 0.0128 - acc: 0.9994 - val_loss: 9.5642 - val_acc: 0.0645
Epoch 6/10
8177/8177 [==============================] - 4s 533us/step - loss: 0.0115 - acc: 0.9994 - val_loss: 9.5956 - val_acc: 0.0645
Epoch 7/10
8177/8177 [==============================] - 4s 534us/step - loss: 0.0100 - acc: 0.9995 - val_loss: 9.6620 - val_acc: 0.0621


#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [0]:
data_gen = ImageDataGenerator(validation_split=0.2, 
                          rescale=1/255,
                          samplewise_center=True, # set input mean to 0 over the sample
                          samplewise_std_normalization=True,  # divide inputs by std of the sample
                          rotation_range=90,       # randomly rotate images in the range (degrees, 0 to 180)
                          width_shift_range=0.2,   # randomly shift images horizontally (fraction of total width)
                          height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                          fill_mode='reflect',     # filling the area outside
                          zoom_range=0.4,          # random zoom
                          horizontal_flip=True,    # randomly flip images
                          vertical_flip=True)      # randomly flip images

In [0]:
train_generator = data_gen.flow(X_train, y_train, batch_size=128)
val_generator = data_gen.flow(X_val,y_val,batch_size = 128)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [51]:
output_imggen = model1.fit_generator(train_generator, epochs=10, validation_data=val_generator, verbose = 2, steps_per_epoch=20, validation_steps=80)


Epoch 1/10
 - 38s - loss: 6.9976 - acc: 0.0248 - val_loss: 5.9700 - val_acc: 0.0165
Epoch 2/10
 - 35s - loss: 4.7921 - acc: 0.0215 - val_loss: 5.0160 - val_acc: 0.0174
Epoch 3/10
 - 36s - loss: 4.7475 - acc: 0.0211 - val_loss: 4.8348 - val_acc: 0.0177
Epoch 4/10
 - 38s - loss: 4.7138 - acc: 0.0199 - val_loss: 4.8259 - val_acc: 0.0200
Epoch 5/10
 - 34s - loss: 4.7044 - acc: 0.0180 - val_loss: 4.7733 - val_acc: 0.0193
Epoch 6/10
 - 34s - loss: 4.6859 - acc: 0.0201 - val_loss: 4.7418 - val_acc: 0.0229
Epoch 7/10
 - 35s - loss: 4.6180 - acc: 0.0281 - val_loss: 4.7375 - val_acc: 0.0240
Epoch 8/10
 - 35s - loss: 4.6343 - acc: 0.0305 - val_loss: 4.7179 - val_acc: 0.0254
Epoch 9/10
 - 36s - loss: 4.6310 - acc: 0.0290 - val_loss: 4.6991 - val_acc: 0.0266
Epoch 10/10
 - 35s - loss: 4.6136 - acc: 0.0273 - val_loss: 4.6745 - val_acc: 0.0248


# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import backend as K

#Before prediction
K.clear_session()
# Instantiate the model with the pre-trained weights (no top)
path = '/content/drive/My Drive/R8_CV_Project2/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model= VGG16(weights = path, input_shape=(im_height,im_width,3), include_top=False)

#After prediction
#K.clear_session()


Print the summary of the base_model

In [69]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [70]:
x = base_model.output

x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(120, activation='softmax')(x)

# Creating new model. Please note that this is NOT a Sequential() model.
from keras.models import Model
custom_model = Model(input=base_model.input, output=x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


### Make all the layers in the base_model (VGG16) to be non-trainable

In [71]:
for layer in custom_model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in custom_model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

input_1
False
block1_conv1
False
block1_conv2
False
block1_pool
False
block2_conv1
False
block2_conv2
False
block2_pool
False
block3_conv1
False
block3_conv2
False
block3_conv3
False
block3_pool
False
block4_conv1
False
block4_conv2
False
block4_conv3
False
block4_pool
False
block5_conv1
False
block5_conv2
False
block5_conv3
False
block5_pool
False
flatten_1
False
dense_1
True
dense_2
True
dense_3
True


### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
custom_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [73]:
custom_model.fit_generator(train_generator, epochs=10, validation_data=val_generator, verbose = 2,  steps_per_epoch=128)

Epoch 1/10
 - 57s - loss: 4.7965 - acc: 0.0139 - val_loss: 4.7644 - val_acc: 0.0176
Epoch 2/10
 - 54s - loss: 4.7124 - acc: 0.0248 - val_loss: 4.6952 - val_acc: 0.0210
Epoch 3/10
 - 54s - loss: 4.6273 - acc: 0.0367 - val_loss: 4.6177 - val_acc: 0.0372
Epoch 4/10
 - 55s - loss: 4.5093 - acc: 0.0504 - val_loss: 4.5243 - val_acc: 0.0523
Epoch 5/10
 - 54s - loss: 4.3750 - acc: 0.0673 - val_loss: 4.4157 - val_acc: 0.0553
Epoch 6/10
 - 54s - loss: 4.2475 - acc: 0.0783 - val_loss: 4.3045 - val_acc: 0.0567
Epoch 7/10
 - 54s - loss: 4.1196 - acc: 0.0912 - val_loss: 4.1867 - val_acc: 0.0709
Epoch 8/10
 - 54s - loss: 4.0057 - acc: 0.1043 - val_loss: 4.1148 - val_acc: 0.1007
Epoch 9/10
 - 54s - loss: 3.9189 - acc: 0.1172 - val_loss: 4.0623 - val_acc: 0.0885
Epoch 10/10
 - 54s - loss: 3.8422 - acc: 0.1246 - val_loss: 4.0554 - val_acc: 0.0890
